In [34]:
import numpy as np

from core_components.tiles.base import TileCoordinate, TileTuple

a = TileTuple(([1], []))

In [ ]:
def coordinate_overlap(dimension1: TileTuple, dimension2: TileTuple) -> bool:
    dim1_set1 = set(dimension1[0])
    dim1_set2 = set(dimension2[0])
    dim1_overlap = dim1_set1.is

    dim2_set1 = set(dimension1[1])
    dim2_set2 = set(dimension2[1])
    dim2_overlap = dim2_set1.isdisjoint(dim2_set2)
    
    return not (dim1_overlap or dim2_overlap)  

In [184]:
a = set([1,2,3])
b = set([0,2])

b.issubset(a)

False

In [171]:
a = np.full((5, 5), fill_value=False, dtype=bool)
a[mask_array] = True
a

array([[ True, False, False, False, False],
       [False, False, False, False, False],
       [False, False, False, False, False],
       [False, False, False, False, False],
       [False, False, False, False, False]])

In [175]:
a &= mask_array
a

array([[False, False, False, False, False],
       [False,  True, False, False, False],
       [False, False, False, False, False],
       [False, False, False, False, False],
       [False, False, False, False, False]])

In [88]:

dim1 = TileTuple((tiles[1][0].tolist()[1:3], tiles[1][0].tolist()[1:4]))
dim2 = TileTuple(([3,4], [3,4]))
coordinate_overlap(dim1, dim2)

False

In [12]:
point1 = (1, 2)
point2 = (1, 3)


dim1 = ([point1[0]], [point1[1]])
dim2 = ([point2[0]], [point2[1]])

overlap(dim1, dim2)

False

In [33]:
point1 = (0, 0)
point2 = (3, 4)
point3 = (4, 4)
point4 = (6, 8)


a = (range(point1[0], point2[0]), range(point1[1], point2[1]))
b = (range(point3[0], point4[0]), range(point3[1], point4[1]))

overlap(a, b)

False

In [52]:
point1 = (1, 1)
point2 = (3, 3)
point3 = (4, 4)
point4 = (6, 6)

a = np.array([[1, 3], [1, 3]])  
b = np.array([[4, 6], [4, 6]])

overlap(a, b)

False

In [53]:
point1 = (1, 1)
point2 = (3, 3)
point3 = (1, 1)
point4 = (3, 3)

a = np.array([[1, 3], [1, 3]])  
b = np.array([[1, 3], [1, 3]])

overlap(a, b)

True

In [61]:
point1 = (1, 1)
point2 = (3, 5)
point3 = (4, 4)
point4 = (5, 5)

a = np.array([np.arange(1,4), np.arange(1,6)])  
b = np.array([np.arange(3, 6), np.arange(3, 6)])

overlap(a, b)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.

In [ ]:
from core_components.generators.random import GenericDungeonGenerator, CIRCULAR_ROOM_TEMPLATE
from core_components.tilemaps.base import MapCoords
import matplotlib.pyplot as plt
import numpy as np
dungeon_gen = GenericDungeonGenerator((50,50))
room_template = CIRCULAR_ROOM_TEMPLATE
center = MapCoords(15, 15)
size = (10, 8)

# Act
instance = dungeon_gen.generate()

In [1]:
from core_components.tilemaps.base import MapCoords
a = MapCoords(14, 15)
b = MapCoords(14, 15)
a == b

True

In [ ]:

plt.imshow(instance.tiles['transparent'])

In [ ]:
dungeon_gen.corridors[0]

In [ ]:
plt.imshow(dungeon_gen.spawn_room(CIRCULAR_ROOM_TEMPLATE, center, size).inner_area)

In [ ]:
mask = np.full((width, height), False)
mask = np.s_[2:5, 2:5]

tile_map._tiles['type'][mask] = tile_map.resources['tiles']['floor']
tile_map._tiles[mask]['type']

In [ ]:
def istype(a: np.ndarray, dtype: np.dtype) -> bool:
    """Helper function to check if a numpy array is of a specific dtype"""
    if a.dtype.metadata is None or dtype.metadata is None:
        return False
    return a.dtype.metadata['__name__'] == dtype.metadata['__name__']

ascii_default = np.array((ord(' '), (255, 255, 255), (0, 0, 0)), dtype=ascii_graphic)

istype(ascii_default, ascii_graphic)  # True

In [ ]:
def run_tests(test_list):
    count = 0
    for test in test_list:
        try:
            test()
            count += 1
        except Exception as e:
            print(f"Exception: {test.__name__}: {e}")
            continue
    print(f"{count} of {len(test_list)} tests ran successfully.")

In [ ]:
from test.test_component_actions_system_dispatcher import *
tests = [test_dispatcher_ev_gamestart,
         test_dispatcher_ev_gameover,
         test_dispatcher_ev_keydown]

run_tests(tests)
# --- IGNORE ---

In [ ]:
from test.test_component_actions_dispatchers import *
tests = [test_dispatcher_ev_quit,
         test_dispatcher_initialization]

run_tests(tests)
# --- IGNORE ---

In [ ]:
from test.test_game_ai import *
test_game_over_event_handling()
test_game_ai_initialization()
test_game_ai_no_events()
test_game_ai_multiple_events()
test_game_ai_event_clearing()



In [ ]:
from test.test_game_map import *
test_get_map_coords()
test_in_bounds()
test_is_traversable_transparent_visible_explored()
test_out_of_bounds_checks()
test_set_visible_and_reset()
test_update_explored()
test_width_height()
test_tiles_initialization()


In [ ]:
from test.test_roster import *
test_roster_spawn()
test_roster_entity_blocked_locations()
test_roster_entity_collision()
test_roster_live_ai_actors()
test_roster_live_actors()
test_roster_get_entity_at_location()
